# Data Loading and Model Testing
This is a streamlined file for quickly loading data and testing out some models.

In [25]:
import numpy as np
import nibabel as nib 
import matplotlib.pyplot as plt 
import tensorflow as tf 
import os 
import pyvista as pv
from sklearn.model_selection import train_test_split
from scipy import ndimage
import random 
from tensorflow import keras
from keras import layers

### Checking for GPU Support 

In [26]:
print(tf.__version__)
if len(tf.config.list_physical_devices('GPU')) > 0:
    print("Using GPU")
else:
    print("Using CPU")


2.10.0
Using GPU


### Data Loading Function 

In [27]:
def load_data(top_dir, test_size=0.2, val_size=0.1):
    # Get a list of all the file paths
    file_paths = []
    labels = []
    for i, folder in enumerate(os.listdir(top_dir)):
        for file in os.listdir(top_dir + '/' + str(folder)):
            if 'max' in str(file):
                file_paths.append(os.path.join(top_dir, folder, file))
                labels.append(i)

    # Load the images into a numpy array
    data = []
    for filename in file_paths:
        img = nib.load(filename)
        data.append(img.get_fdata())
    data = np.array(data)

    # Convert labels to numpy array
    labels = np.array(labels)

    # Reassign labels assigned to 2 to 0
    labels[labels==2] = 0

    # Split the data into train, validation, and test sets
    X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=test_size, stratify=labels)
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=val_size, stratify=y_train)

    return X_train, X_val, X_test, y_train, y_val, y_test

### Loading Data 

In [28]:
# Loading our data
X_train, X_val, X_test, y_train, y_val, y_test = load_data('../InSpect/datasets')
print("Finished Loading Data!")


Finished Loading Data!


### Dealing With GPU Issues 
For some reason, if I try to load the following dataset to memory I can't, so I'll try limiting the memory to see if that works. 

In [29]:
'''
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    # Restrict TensorFlow to only allocate 2GB * 2 of memory on the first GPU
    try:
        tf.config.experimental.set_virtual_device_configuration(
            gpus[0],
            [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=2048 * 2)])
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Virtual devices must be set before GPUs have been initialized
        print(e)
'''

'\ngpus = tf.config.experimental.list_physical_devices(\'GPU\')\nif gpus:\n    # Restrict TensorFlow to only allocate 2GB * 2 of memory on the first GPU\n    try:\n        tf.config.experimental.set_virtual_device_configuration(\n            gpus[0],\n            [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=2048 * 2)])\n        logical_gpus = tf.config.experimental.list_logical_devices(\'GPU\')\n        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")\n    except RuntimeError as e:\n        # Virtual devices must be set before GPUs have been initialized\n        print(e)\n'

### Data Processing 

In [30]:
'''
def train_preprocessing(x, y):
    x = tf.image.random_flip_left_right(x)
    x = tf.image.resize(x, [256, 256])
    x = tf.image.random_crop(x, [224, 224, 3])
    x = tf.keras.applications.resnet_v2.preprocess_input(x)
    y = tf.one_hot(y, depth=5)
    return x, y

def validation_preprocessing(x, y):
    x = tf.image.resize(x, [224, 224])
    x = tf.keras.applications.resnet_v2.preprocess_input(x)
    y = tf.one_hot(y, depth=5)
    return x, y


train_loader = tf.data.Dataset.from_tensor_slices((X_train, y_train))
validation_loader = tf.data.Dataset.from_tensor_slices((X_val, y_val))

#batch_size = 2


train_dataset = (
    train_loader.shuffle(len(X_train))
    .map(lambda x, y: train_preprocessing(x, y))
    .batch(batch_size)
    .prefetch(2)
)

validation_dataset = (
    validation_loader.shuffle(len(X_val))
    .map(lambda x, y: validation_preprocessing(x, y))
    .batch(batch_size)
    .prefetch(2)
)
'''

'\ndef train_preprocessing(x, y):\n    x = tf.image.random_flip_left_right(x)\n    x = tf.image.resize(x, [256, 256])\n    x = tf.image.random_crop(x, [224, 224, 3])\n    x = tf.keras.applications.resnet_v2.preprocess_input(x)\n    y = tf.one_hot(y, depth=5)\n    return x, y\n\ndef validation_preprocessing(x, y):\n    x = tf.image.resize(x, [224, 224])\n    x = tf.keras.applications.resnet_v2.preprocess_input(x)\n    y = tf.one_hot(y, depth=5)\n    return x, y\n\n\ntrain_loader = tf.data.Dataset.from_tensor_slices((X_train, y_train))\nvalidation_loader = tf.data.Dataset.from_tensor_slices((X_val, y_val))\n\n#batch_size = 2\n\n\ntrain_dataset = (\n    train_loader.shuffle(len(X_train))\n    .map(lambda x, y: train_preprocessing(x, y))\n    .batch(batch_size)\n    .prefetch(2)\n)\n\nvalidation_dataset = (\n    validation_loader.shuffle(len(X_val))\n    .map(lambda x, y: validation_preprocessing(x, y))\n    .batch(batch_size)\n    .prefetch(2)\n)\n'

### Model Definition

In [31]:
def get_model(width=79, height=95, depth=79):
    """Build a 3D convolutional neural network model."""

    inputs = keras.Input((width, height, depth, 1))

    x = layers.Conv3D(filters=64, kernel_size=3, activation="relu")(inputs)
    x = layers.MaxPool3D(pool_size=2)(x)
    x = layers.BatchNormalization()(x)

    x = layers.Conv3D(filters=64, kernel_size=3, activation="relu")(x)
    x = layers.MaxPool3D(pool_size=2)(x)
    x = layers.BatchNormalization()(x)

    x = layers.Conv3D(filters=128, kernel_size=3, activation="relu")(x)
    x = layers.MaxPool3D(pool_size=2)(x)
    x = layers.BatchNormalization()(x)

    x = layers.Conv3D(filters=256, kernel_size=3, activation="relu")(x)
    x = layers.MaxPool3D(pool_size=2)(x)
    x = layers.BatchNormalization()(x)

    x = layers.GlobalAveragePooling3D()(x)
    x = layers.Dense(units=512, activation="relu")(x)
    x = layers.Dropout(0.3)(x)

    outputs = layers.Dense(units=1, activation="sigmoid")(x)

    # Define the model.
    model = keras.Model(inputs, outputs, name="3dcnn")
    return model


### Quick X-train EDA 
It doesn't like the way its being passed the training and validation data. Let's see what's in there and try to give it a chance to train on GPU. If it works on this side, that means the gpu data modification has a problem only for data loading. Which we can fix by generating the tf.Dataset object on CPU then saving it in a file for training on the GPU. 

### Conclusion
Oops, since I cut out those earlier functions. I need to re-integrate y-train into the equation. 

In [32]:
print(np.shape(X_train))
print(np.shape(y_train))

(743, 79, 95, 79)
(743,)


### Building and Compiling Model 
I have now spent a great deal of time and energy getting myself to a point where I now see that my current model will not fit on my GPU, this transfer learning has moved from a potential upside to an absolute necessity. Good thing I was already planning on doing that! On the plus side, since I was already planning on using CPU for my data pre-precessing, I won't have to make any significant changes and I already have everything in the form of a tf.Dataset! Sometimes you gotta take the good with the bad!

In [33]:
# Buld Model
model = get_model()

# Compile model
initial_learning_rate = 0.0001
lr_schedule = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate, decay_steps=100000, decay_rate=0.96, staircase=True
)
model.compile(
    loss="binary_crossentropy",
    optimizer=keras.optimizers.Adam(learning_rate=lr_schedule),
    metrics=["acc"],
)

'''
# Define callbacks.
checkpoint_cb = keras.callbacks.ModelCheckpoint(
    "3d_image_classification.h5", save_best_only=True
)
early_stopping_cb = keras.callbacks.EarlyStopping(monitor="val_acc", patience=15)
'''


# Train the model, doing validation at the end of each epoch
epochs = 5
model.fit(
    x=X_train,
    y=y_train,
    #validation_data=X_val,
    epochs=epochs,
    shuffle=True,
    #callbacks=[checkpoint_cb, early_stopping_cb],
)

ResourceExhaustedError: {{function_node __wrapped__Mul_device_/job:localhost/replica:0/task:0/device:GPU:0}} failed to allocate memory [Op:Mul]